<h1><center>Programming Exercise 8:<br>Anomaly Detection and Recommender Systems</center></h1>

# 1. Anomaly Detection

## 1.1 Gaussian Distribution

## 1.2 Estimating Parameters for a Gaussian

## 1.3 Selecting the Threshold $\epsilon$

## 1.4 High-dimensioanl Dataset

# 2. Recommender Systems

## 2.1 Movie Ratings Dataset

## 2.2 Collaborative Filtering Algorithm

## 2.3 Learning Movie Recommendations